## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-24-05-49-30 +0000,nyt,"Jimmy Kimmel, Somber but Defiant, Defends Free...",https://www.nytimes.com/2025/09/23/business/me...
1,2025-09-24-05-48-59 +0000,nyt,Delays Expected at London Stansted Airport Aft...,https://www.nytimes.com/2025/09/24/world/europ...
2,2025-09-24-05-43-32 +0000,bbc,Cost of living charities warn of rising bills,https://www.bbc.com/news/articles/cp8jepmepy1o...
3,2025-09-24-05-31-25 +0000,nypost,"Elderly Kansas City worker, 66, ruthlessly att...",https://nypost.com/2025/09/24/us-news/kansas-c...
4,2025-09-24-05-28-35 +0000,nyt,"Like His Suspension, Kimmel’s Return Draws a P...",https://www.nytimes.com/2025/09/24/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,77
1,kimmel,20
0,jimmy,15
29,was,14
240,un,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
44,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/the-franti...,156
19,2025-09-24-04-01-42 +0000,nypost,"Defiant, emotional Jimmy Kimmel returns to the...",https://nypost.com/2025/09/24/us-news/defiant-...,149
151,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...,135
6,2025-09-24-05-04-00 +0000,wsj,Kimmel Makes Emotional Return to ABC With Trum...,https://www.wsj.com/business/media/jimmy-kimme...,134
274,2025-09-23-12-21-27 +0000,nypost,Trump to tout US as defender of Western civili...,https://nypost.com/2025/09/23/us-news/trump-to...,127


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
44,156,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/the-franti...
230,79,2025-09-23-15-26-23 +0000,latimes,Nexstar and Sinclair TV stations will not run ...,https://www.latimes.com/entertainment-arts/bus...
182,55,2025-09-23-17-53-00 +0000,wsj,Fed Chair Jerome Powell said he judged the Fed...,https://www.wsj.com/economy/central-banking/fe...
151,53,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...
15,51,2025-09-24-04-15-00 +0000,wsj,China said Tuesday it will no longer seek new ...,https://www.wsj.com/economy/trade/china-to-sto...
274,46,2025-09-23-12-21-27 +0000,nypost,Trump to tout US as defender of Western civili...,https://nypost.com/2025/09/23/us-news/trump-to...
195,41,2025-09-23-17-29-24 +0000,nypost,Megyn Kelly slams ‘coward’ Van Jones for brand...,https://nypost.com/2025/09/23/media/megyn-kell...
33,35,2025-09-24-02-36-48 +0000,nypost,24-year-old illegal immigrant who posed as Ohi...,https://nypost.com/2025/09/23/us-news/24-year-...
133,34,2025-09-23-20-37-00 +0000,wsj,Trump backed out of a meeting with Democrats a...,https://www.wsj.com/politics/policy/trump-demo...
147,31,2025-09-23-19-52-00 +0000,wsj,"Canada Needs to Chart New Economic Course, Ban...",https://www.wsj.com/articles/canada-needs-to-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
